In [1]:


import pandas as pd

df = pd.read_csv('./reverse_image_search.csv')
df.head()



,id,path,label
0,0,./train/brain_coral/n01917289_1783.JPEG,brain_coral
1,1,./train/brain_coral/n01917289_4317.JPEG,brain_coral
2,2,./train/brain_coral/n01917289_765.JPEG,brain_coral
3,3,./train/brain_coral/n01917289_1079.JPEG,brain_coral
4,4,./train/brain_coral/n01917289_2484.JPEG,brain_coral


In [2]:


import cv2
from towhee.types.image import Image

id_img = df.set_index('id')['path'].to_dict()
def read_images(results):
    imgs = []
    for re in results:
        path = id_img[re.id]
        imgs.append(Image(cv2.imread(path), 'BGR'))
    return imgs



In [3]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

def create_milvus_collection(collection_name, dim):
    connections.connect(host='127.0.0.1', port='19530')
    
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
    FieldSchema(name='path', dtype=DataType.VARCHAR, description='path to image', max_length=500, 
                    is_primary=True, auto_id=False),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, description='embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='text image search')
    collection = Collection(name=collection_name, schema=schema)

    # create IVF_FLAT index for collection.
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":512}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection

#collection = create_milvus_collection('text_image_search', 512)

In [4]:
from pymilvus import Collection, utility
connections.connect(host='127.0.0.1', port='19530')

collection = create_milvus_collection('text_image_search', 512)

In [5]:


from towhee import ops, pipe, DataCollection
import numpy as np



In [7]:


p = (
    pipe.input('path')
    .map('path', 'img', ops.image_decode.cv2('rgb'))
    .map('img', 'vec', ops.image_text_embedding.clip(model_name='clip_vit_base_patch16', modality='image'))
    .map('vec', 'vec', lambda x: x / np.linalg.norm(x))
    .output('img', 'vec')
)

DataCollection(p('./teddy.png')).show()



2024-04-02 20:34:24,111 - 140377342342720 - image_decode_cv2.py-image_decode_cv2:68 - ERROR: Read image ./teddy.png failed
[ WARN:0@5.099] global loadsave.cpp:244 findDecoder imread_('./teddy.png'): can't open/read file: check file path/integrity


RuntimeError: Node-image-decode/cv2-0 runs failed, error msg: Read image ./teddy.png failed, Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/towhee/runtime/nodes/node.py", line 156, in _call
    return True, self._op(*inputs), None
  File "/home/kpamuk/.towhee/operators/image-decode/cv2/versions/main/image_decode_cv2.py", line 69, in __call__
    raise RuntimeError(err)
RuntimeError: Read image ./teddy.png failed



In [10]:


p2 = (
    pipe.input('text')
    .map('text', 'vec', ops.image_text_embedding.clip(model_name='clip_vit_base_patch16', modality='text'))
    .map('vec', 'vec', lambda x: x / np.linalg.norm(x))
    .output('text', 'vec')
)

DataCollection(p2("cat")).show()



text,vec
cat,"[0.03314916, -0.020378197, 0.0001162347, ...] shape=(512,)"


In [12]:
p = DataCollection(p2("A teddybear on a skateboard in Times Square."))


In [13]:
%%time
collection = create_milvus_collection('text_image_search', 512)

def read_csv(csv_path, encoding='utf-8-sig'):
    import csv
    with open(csv_path, 'r', encoding=encoding) as f:
        data = csv.DictReader(f)
        for line in data:
            yield int(line['id']), line['path']

p3 = (
    pipe.input('csv_file')
    .flat_map('csv_file', ('id', 'path'), read_csv)
    .map('path', 'img', ops.image_decode.cv2('rgb'))
    .map('img', 'vec', ops.image_text_embedding.clip(model_name='clip_vit_base_patch16', modality='image', device=0))
    .map('vec', 'vec', lambda x: x / np.linalg.norm(x))
    .map(('path', 'vec'), (), ops.ann_insert.milvus_client(host='127.0.0.1', port='19530', collection_name='text_image_search'))
    .output()
)

ret = p3('reverse_image_search.csv')

CPU times: user 24 s, sys: 6.38 s, total: 30.4 s
Wall time: 17.2 s


In [17]:
collection.load()

In [18]:
print('Total number of inserted data is {}.'.format(collection.num_entities))

Total number of inserted data is 0.


In [21]:
import pandas as pd
import cv2

def read_image(image_ids):
    df = pd.read_csv('reverse_image_search.csv')
    id_img = df.set_index('id')['path'].to_dict()
    imgs = []
    decode = ops.image_decode.cv2('rgb')
    for image_id in image_ids:
        path = id_img[image_id]
        imgs.append(decode(path))
    return imgs


p4 = (
    pipe.input('text')
    .map('text', 'vec', ops.image_text_embedding.clip(model_name='clip_vit_base_patch16', modality='text'))
    .map('vec', 'vec', lambda x: x / np.linalg.norm(x))
    .map('vec', 'result', ops.ann_search.milvus_client(host='127.0.0.1', port='19530', collection_name='text_image_search', limit=5))
    .map('result', 'image_ids', lambda x: [item[0] for item in x])
    .map('image_ids', 'images', read_image)
    .output('text', 'images')
)


In [ ]:

DataCollection(p4("A white dog")).show()
DataCollection(p4("A black dog")).show()
